In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import pygeos
from osgeo import ogr,gdal
from tqdm import tqdm
gdal.SetConfigOption("OSM_CONFIG_FILE", os.path.join('..',"osmconf.ini"))
from pygeos import from_wkb


In [2]:
def query_b(geoType,keyCol,**valConstraint):
    """
    This function builds an SQL query from the values passed to the retrieve() function.
    Arguments:
         *geoType* : Type of geometry (osm layer) to search for.
         *keyCol* : A list of keys/columns that should be selected from the layer.
         ***valConstraint* : A dictionary of constraints for the values. e.g. WHERE 'value'>20 or 'value'='constraint'
    Returns:
        *string: : a SQL query string.
    """
    query = "SELECT " + "osm_id"
    for a in keyCol: query+= ","+ a  
    query += " FROM " + geoType + " WHERE "
    # If there are values in the dictionary, add constraint clauses
    if valConstraint: 
        for a in [*valConstraint]:
            # For each value of the key, add the constraint
            for b in valConstraint[a]: query += a + b
        query+= " AND "
    # Always ensures the first key/col provided is not Null.
    query+= ""+str(keyCol[0]) +" IS NOT NULL" 
    return query 


def retrieve(osm_path,geoType,keyCol,**valConstraint):
    """
    Function to extract specified geometry and keys/values from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.     
        *geoType* : Type of Geometry to retrieve. e.g. lines, multipolygons, etc.
        *keyCol* : These keys will be returned as columns in the dataframe.
        ***valConstraint: A dictionary specifiying the value constraints.  
        A key can have multiple values (as a list) for more than one constraint for key/value.  
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all columns, geometries, and constraints specified.    
    """
    driver=ogr.GetDriverByName('OSM')
    data = driver.Open(osm_path)
    query = query_b(geoType,keyCol,**valConstraint)
    sql_lyr = data.ExecuteSQL(query)
    features =[]
    # cl = columns 
    cl = ['osm_id'] 
    for a in keyCol: 
        cl.append(a)
    if data is not None:
        print('query is finished, lets start the loop')
        for feature in tqdm(sql_lyr,desc='extract'):
            try:
                if feature.GetField(keyCol[0]) is not None:
                    geom = pygeos.from_wkt(feature.geometry().ExportToWkt()) 
                    if geom is None:
                        continue
                    # field will become a row in the dataframe.
                    field = []
                    for i in cl: field.append(feature.GetField(i))
                    field.append(geom)   
                    features.append(field)
            except:
                print("WARNING: skipped OSM feature")   
    else:
        print("ERROR: Nonetype error when requesting SQL. Check required.")    
    
    cl.append('geometry')                   
    if len(features) > 0:
        return pd.DataFrame(features,columns=cl)
    else:
        print("WARNING: No features or No Memory. returning empty GeoDataFrame") 
        return pd.DataFrame(columns=['osm_id','geometry'])

def buildings(osm_path):
    """
    Function to extract building polygons from OpenStreetMap    
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique building polygons.    
    """
    return retrieve(osm_path, 'multipolygons',['building'])

In [3]:
provinces = [x for x in os.listdir(os.path.join("C:\\Data",'OSM','PBF')) if 'netherlands' not in x]

In [16]:
provinces[11]

'zuid-holland-latest.osm.pbf'

In [17]:
#for province in provinces:
province = provinces[11]
osm_path = os.path.join("C:\\Data",'OSM','PBF',province)
df = buildings(osm_path)
df = df.drop('osm_id',axis=1)

province_name = province.split("-")[0]+'_'+province.split("-")[1]
df.geometry = pygeos.to_wkb(df.geometry.values)
df.to_feather(os.path.join("C:\\Data",'OSM','feather',"{}.ft".format(province_name)))

query is finished, lets start the loop


extract: 100%|█████████████████████████████████████████████████████████████| 1803281/1803281 [07:30<00:00, 4005.97it/s]


In [10]:
provinces[5]

'limburg-latest.osm.pbf'

In [15]:
df

,building,geometry
0,house,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
1,yes,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
2,yes,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
3,yes,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
4,commercial,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
...,...,...
1448314,construction,"b""\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03..."
1448315,yes,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
1448316,yes,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
1448317,yes,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
